In [1]:
import pygame
import random
import math
import numpy as np 
from collections import (deque, defaultdict)

# 常量，屏幕宽高
WIDTH, HEIGHT = 800, 800
RADIUS = 30

# 初始化操作
pygame.init()
pygame.mixer.init()
# 创建游戏窗口
screen = pygame.display.set_mode((WIDTH, HEIGHT))

# 设置游戏标题
pygame.display.set_caption('妹多局之卡坦岛')

pygame 2.3.0 (SDL 2.24.2, Python 3.7.3)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [2]:
a = defaultdict(int)

In [3]:
a['a'] += 1
a

defaultdict(int, {'a': 1})

In [4]:
rect = pygame.rect.Rect(0, 0, 10, 20)

In [5]:
rect.top

0

In [6]:
import itertools
a = list('abc')
b = list('12')

for t in zip(a, b):
    print(t)


('a', '1')
('b', '2')


In [7]:
# red, blue, yellow, green, purple = [pygame.Color(color) for color in ['red', 'blue', 'yellow', 'green', 'purple']]

In [8]:
def draw_colorized_contour(rect, width, color):
    for start, end in [
        (rect.topleft, rect.topright), 
        (rect.topright, rect.bottomright), 
        (rect.bottomleft, rect.bottomright),
        (rect.topleft, rect.bottomleft)
    ]:
        pygame.draw.line(screen, color, start, end, width)

In [ ]:
# 添加系统时钟，用于设置帧的刷新
FPS = 10
clock = pygame.time.Clock()

In [ ]:
# class Text(pygame.sprite.Sprite):
#     def __init__(self, text, center, size=20, color=blue, width=10, height=10):
#         super().__init__()
    
#         self.font = pygame.font.SysFont("Arial", size)
#         self.text_surf = self.font.render(text, 1, color)
#         self.image = pygame.Surface((width, height))
#         W = self.text_surf.get_width()
#         H = self.text_surf.get_height()
#         self.rect = self.image.get_rect()
#         self.image.blit(self.text_surf, center)
        
        
class Edge():
    WIDTH = 5 
    def __init__(self, start_pos, end_pos, color='yellow'):
        super().__init__()
        self.start_pos, self.end_pos = start_pos, end_pos
        pygame.draw.line(screen, pygame.Color(color), start_pos, end_pos, self.WIDTH)
    
    def update(self, color):
        pygame.draw.line(screen, pygame.Color(color), self.start_pos, self.end_pos, self.WIDTH)

In [ ]:
# 定义class
class Vertex(pygame.sprite.Sprite):
    WIDTH, HEIGHT = 15, 15
    
    def __init__(self, x, y, color=None):
        super().__init__()
        self.center = (x, y)
        self.rect = pygame.rect.Rect(x, y, self.WIDTH, self.HEIGHT)
        self.image = pygame.image.load("./images/catan/empty.jpeg")
        self.image = pygame.transform.scale(self.image, (self.WIDTH, self.HEIGHT))
        self.color = color
    
    def update(self, color):
        # colorize the contour
        _color = pygame.Color(color)
        draw_colorized_contour(rect=self.rect, width=int(self.WIDTH / 10), color=_color)
        
#         colorImage = pygame.Surface(self.image.get_size()).convert_alpha()
#         colorImage.fill(color)
#         self.image.blit(colorImage, self.rect.center, special_flags = pygame.BLEND_RGBA_MULT)

In [ ]:
class Player(pygame.sprite.Sprite):
    """player
    """
    WIDTH, HEIGHT = 30, 30
    ROAD = ['brick', 'lumber']
    VILLAGE = ROAD + ['wheat', 'wool']
    
    def __init__(self, color, x, y, name):
        super().__init__()
        self.color = color 
        self.image = pygame.image.load("./images/catan/player.jpeg")
        self.rect = pygame.rect.Rect(x, y, self.WIDTH, self.HEIGHT)
        self.image = pygame.transform.scale(self.image, (self.WIDTH, self.HEIGHT))
        self.resources = defaultdict(int)
        self.name = name
        self.font = pygame.font.SysFont('Arial', 25)
        
    def roll_dice(self):
        return random.randint(1, 6) + random.randint(1, 6) 
    
    def build_road(self):
        if all([self.resources.get(res, 0) >= 1 for res in self.ROAD]):
            for res in self.ROAD:
                self.resources[res] -= 1
            return True 
        return False 
    
    def build_village(self):
        if all([self.resources.get(res, 0) >= 1 for res in self.VILLAGE]):
            for res in self.VILLAGE:
                self.resources[res] -= 1
            return True 
        return False 
    
    def show_card(self):
        draw_colorized_contour(rect=self.rect, width=int(self.WIDTH / 10), color=pygame.Color(self.color))
        
        screen.blit(
            source=self.font.render(
                ",".join([f"我是ikun,我的资源是{k}:{v}" for k,v in self.resources.items()]), 
                True, 
                pygame.Color(self.color)
            ), 
            dest=(self.rect.x + 2 * self.WIDTH, self.rect.y)
        )
            

In [ ]:
class Hexagon(pygame.sprite.Sprite):
    """hexagonal tile
    
    Attrs:
        no: rolling number
        res: resource type, options ['wool', 'lumber', 'wheat', 'ore', 'brick', 'desert']
        vertices: six vertices, where players can build settlements
        edges: six lines, where players can build road
        colors: list of rgb-based triple, representing the owners of each vertex
    """
    RADIUS = RADIUS 
    RESOURCE_TYPES = 'wool lumber wheat ore brick desert'.split(' ')
    RESOURCE_IMAGES = [pygame.image.load(f"./images/catan/{res}.jpeg") for res in RESOURCE_TYPES]
    TIPO2IMAGE = dict(zip(RESOURCE_TYPES, RESOURCE_IMAGES))
    G3 = math.sqrt(3)
    WIDTH, HEIGHT = RADIUS * 2 * G3 - 2 * Edge.WIDTH, 2 * RADIUS - 2 * Edge.WIDTH
    
    def __init__(self, no, resource_type, center_x, center_y, letter=''):
        super().__init__()
        self.no = no
        self.center_x, self.center_y = center_x, center_y
        self.res = resource_type
        self.image = self.TIPO2IMAGE[self.res]
        self.image = pygame.transform.scale(self.image, (self.WIDTH, self.HEIGHT))
        self.rect = pygame.rect.Rect(center_x - self.WIDTH / 2, center_y - self.RADIUS, self.WIDTH, self.HEIGHT)
        self.font = pygame.font.SysFont('Arial', 25)
        self.colors = ['yellow' for _ in range(6)]
        
        angles = [i * math.pi / 3 + math.pi / 6 for i in range(6)]
        
        self.vertices = [
            Vertex(
                x=self.center_x + self.RADIUS * 2 * math.cos(angle) - Vertex.WIDTH / 2,
                y=self.center_y + self.RADIUS * 2 * math.sin(angle) + Vertex.HEIGHT / 2,
                color=color
            ) 
            for angle, color in zip(angles, self.colors)
        ]

        self.edges = None
        self.letter = letter
        
    def update(self):
        pass 
    
    def draw_edges(self):
        self.edges = [
            Edge(
                start_pos=self.vertices[i].center, 
                end_pos=self.vertices[(i+1) % 6].center,
                color=self.colors[i]
            )
            for i in range(6)
        ]
        
    def change_edge_color(self, i, color):
        edge = self.edges[i]
        self.colors[i] = pygame.Color(color)
        pygame.draw.line(screen, color, edge.start_pos, edge.end_pos, 5)
        
    def show_no(self, color='red'):
        """显示在六边形的上半部"""
        screen.blit(
            self.font.render(f"{self.letter}:{self.no}", True, pygame.Color(color)), 
            (self.rect.x + self.G3 * self.RADIUS, self.rect.y - self.RADIUS / 2)
        )

In [ ]:
# 初始化精灵组
hex_group = pygame.sprite.Group()
player_group = pygame.sprite.Group()


# # 初始化vertices
# ves = [Vertex(100, 100)]
# ver_group.add(ves[0])

# 随机点数和资源
no_list = [i for i in range(2, 13) if i != 7] * 3 
for i in [2, 12]:
    no_list.remove(i)

resource_list = ['lumber', 'wool', 'wheat'] * 6 + ['ore', 'brick'] * 5 
random.shuffle(resource_list)
init_layout = list(zip(no_list, resource_list))
init_layout += [(7, 'desert')] * 2
random.shuffle(init_layout)


# 初始化背景
# x_unit和y_unit表示相邻两层的第一个六边形中心的坐标差
x_unit = Hexagon.G3 * Hexagon.RADIUS
y_unit = Hexagon.RADIUS * 3

hexs = list()
no2tiles = dict()

for i in range(-3, 4):
    first_x, first_y = x_unit * (abs(i) + 1),  y_unit * (i + 4)
    for j in range(6 - abs(i)):
        tile = init_layout.pop()
        hexagon = Hexagon(
            no=tile[0],
            center_x=first_x + 2 * x_unit * j,
            center_y=first_y,
            resource_type=tile[1]
        )
        hexs.append(hexagon)
        no2tiles.setdefault(tile[0], []).append(hexagon)

        
letters = list('badmintonkg')
letter2tile = dict()
for tile, letter in zip(hexs, letters):
    tile.letter = letter
    letter2tile[letter] = tile
        

for ele in hexs:
    ele.draw_edges()
    hex_group.add(ele)

names = ['小蓝', '小紫', '小红', '小绿']
colors = ['blue', 'purple', 'red', 'green']
xs = [0, 0, HEIGHT - Player.HEIGHT, HEIGHT - Player.HEIGHT]
ys = [0, WIDTH - Player.WIDTH] * 2
players = [Player(color=color, x=x, y=y, name=name) for x, y, color, name in zip(xs, ys, colors, names)]
players = deque(players)

# for player in players:
#     player_group.add(player)

color2player = dict([(player.color, player) for player in players])

In [ ]:
# 初始化玩家资源
# TODO

In [ ]:
# 保持游戏运行状态(游戏循环）
i = 1
while players:
    player = players.popleft()
    players.append(player)
    
    # roll点
    no = player.roll_dice()      
    tiles = no2tiles[no]
    for tile in tiles:
        resource = tile.res
        for vertex in tile.vertices:
            if vertex.color is not None:
                if vertex.color in color2player:
                    jugador = color2player[vertex.color]
                    jugador.resources[resource] += 1
                
    print(f"{i}轮摇出来{no}")
    for joueur in players:
        print(f"{joueur.name}: {joueur.resources}")
        
    while True:
        # 检测事件
        for event in pygame.event.get():
            # 检测关闭按钮被点击的事件
            if event.type == pygame.QUIT:
                # 退出
                pygame.quit()
                exit()
                
        screen.fill((100, 100, 100))

        for group in [hex_group]:
            group.update()
            group.draw(screen)

        
        keys = pygame.key.get_pressed()
            
#         # 是否要修建
#         if keys[pygame.K_b]:
#             ## TODO
#             # 选tile
#             if keys[pygame.K_r]:
                
        
        # 修改边的颜色
        hexs[0].change_edge_color(2, 'red')

        for tile in hexs:
            tile.draw_edges()
            tile.show_no()
            ver_group = pygame.sprite.Group()
            for vertex in tile.vertices:
                ver_group.add(vertex)
            ver_group.draw(screen)
        
        for player in players:
            player.show_card()

        pygame.display.update()
        
        # pass over to the next player
        if keys[pygame.K_0]:
            i += 1
            break 

1轮摇出来8
小紫: defaultdict(<class 'int'>, {})
小红: defaultdict(<class 'int'>, {})
小绿: defaultdict(<class 'int'>, {})
小蓝: defaultdict(<class 'int'>, {})
2轮摇出来7
小红: defaultdict(<class 'int'>, {})
小绿: defaultdict(<class 'int'>, {})
小蓝: defaultdict(<class 'int'>, {})
小紫: defaultdict(<class 'int'>, {})
3轮摇出来7
小绿: defaultdict(<class 'int'>, {})
小蓝: defaultdict(<class 'int'>, {})
小紫: defaultdict(<class 'int'>, {})
小红: defaultdict(<class 'int'>, {})
4轮摇出来6
小蓝: defaultdict(<class 'int'>, {})
小紫: defaultdict(<class 'int'>, {})
小红: defaultdict(<class 'int'>, {})
小绿: defaultdict(<class 'int'>, {})
5轮摇出来4
小紫: defaultdict(<class 'int'>, {})
小红: defaultdict(<class 'int'>, {})
小绿: defaultdict(<class 'int'>, {})
小蓝: defaultdict(<class 'int'>, {})
6轮摇出来2
小红: defaultdict(<class 'int'>, {})
小绿: defaultdict(<class 'int'>, {})
小蓝: defaultdict(<class 'int'>, {})
小紫: defaultdict(<class 'int'>, {})
7轮摇出来5
小绿: defaultdict(<class 'int'>, {})
小蓝: defaultdict(<class 'int'>, {})
小紫: defaultdict(<class 'int'>, {})
小红: de

In [ ]:
# # 保持游戏运行状态(游戏循环）
# while True:
#     # ===========游戏帧的刷新===========
#     clock.tick(FPS)

#     # 检测事件
#     for event in pygame.event.get():
#         # 检测关闭按钮被点击的事件
#         if event.type == pygame.QUIT:
#             # 退出
#             pygame.quit()
#             exit()
    
#     screen.fill((100, 100, 100))

#     for group in [hex_group]:
#         group.update()
#         group.draw(screen)
    

#     # 修改边的颜色
#     # 两种方法都可以
# #     for edge in hexs[0].edges:
# #         edge.update(blue)
#     hexs[0].change_edge_color(2, red)
    
#     # 修改第一个格子上角的颜色
#     for vertex in hexs[0].vertices:
#         vertex.update((200, 200, 200))
        
#     for tile in hexs:
#         tile.draw_edges()
#         tile.show_no()
#         ver_group = pygame.sprite.Group()
#         for vertex in tile.vertices:
#             ver_group.add(vertex)
#         ver_group.draw(screen)
            
#     pygame.display.update()